In [69]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [70]:
cups = "WorldCups.csv"
matches="WorldCupMatches.csv"

In [71]:
matches_df = pd.read_csv(matches)
matches_df['Year'] = matches_df['Year'].astype('Int64')
matches_df = matches_df.drop(columns=['Datetime','Home Team Name','Away Team Name','Win conditions','Half-time Home Goals','Half-time Away Goals','RoundID','MatchID','Assistant 1','Assistant 2'])
matches_df = matches_df[matches_df['Stage'] == 'Final']
matches_df= matches_df.rename(columns={'Home Team Goals':'Winner_goals', 'Away Team Goals':'Second_place_goals','Home Team Initials':'Winner_initials','Away Team Initials':'Second_place_initials','Attendance':'Match_attendance'})
matches_df['Match_attendance'] = matches_df['Match_attendance'].astype('Int64')
matches_df['Winner_goals'] = matches_df['Winner_goals'].astype('Int64')
matches_df['Second_place_goals'] = matches_df['Second_place_goals'].astype('Int64')
matches_df.head()

,Year,Stage,Stadium,City,Winner_goals,Second_place_goals,Match_attendance,Referee,Winner_initials,Second_place_initials
17,1930,Final,Estadio Centenario,Montevideo,4,2,68346,LANGENUS Jean (BEL),URU,ARG
34,1934,Final,Nazionale PNF,Rome,2,1,55000,EKLIND Ivan (SWE),ITA,TCH
52,1938,Final,Stade Olympique,Colombes,4,2,45000,CAPDEVILLE Georges (FRA),ITA,HUN
100,1954,Final,Wankdorf Stadium,Berne,3,2,62500,LING William (ENG),FRG,HUN
135,1958,Final,Rasunda Stadium,Solna,5,2,49737,GUIGUE Maurice (FRA),BRA,SWE


In [72]:
cups_df = pd.read_csv(cups)
cups_df = cups_df.drop(columns=['Third','Fourth','Attendance'])
cups_df = cups_df.rename(columns={'Runners-Up':'Second_place', 'GoalsScored':'Goals_scored','QualifiedTeams':'Qualified_teams','MatchesPlayed':'Matches_played'})
cups_df.head()

,Year,Country,Winner,Second_place,Goals_scored,Qualified_teams,Matches_played
0,1930,Uruguay,Uruguay,Argentina,70,13,18
1,1934,Italy,Italy,Czechoslovakia,70,16,17
2,1938,France,Italy,Hungary,84,15,18
3,1950,Brazil,Uruguay,Brazil,88,13,22
4,1954,Switzerland,Germany FR,Hungary,140,16,26


### Connect to local database

In [73]:
rds_connection_string = "postgres:postgres@localhost:5432/ETL"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [74]:
engine.table_names()

['worldcups', 'worldcupmatches']

### Use pandas to load csv converted DataFrame into database

In [75]:
matches_df.to_sql(name='worldcupmatches', con=engine, if_exists='append', index=False)

In [76]:
cups_df.to_sql(name='worldcups', con=engine, if_exists='append', index=False)

In [77]:
pd.read_sql_query('select * from worldcupmatches', con=engine).head()

,Year,Stage,Stadium,City,Winner_goals,Second_place_goals,Match_attendance,Referee,Winner_initials,Second_place_initials
0,1930,Final,Estadio Centenario,Montevideo,4,2,68346,LANGENUS Jean (BEL),URU,ARG
1,1934,Final,Nazionale PNF,Rome,2,1,55000,EKLIND Ivan (SWE),ITA,TCH
2,1938,Final,Stade Olympique,Colombes,4,2,45000,CAPDEVILLE Georges (FRA),ITA,HUN
3,1954,Final,Wankdorf Stadium,Berne,3,2,62500,LING William (ENG),FRG,HUN
4,1958,Final,Rasunda Stadium,Solna,5,2,49737,GUIGUE Maurice (FRA),BRA,SWE


In [78]:
pd.read_sql_query('select * from worldcups', con=engine).head()

,Year,Country,Winner,Second_place,Goals_scored,Qualified_teams,Matches_played
0,1930,Uruguay,Uruguay,Argentina,70,13,18
1,1934,Italy,Italy,Czechoslovakia,70,16,17
2,1938,France,Italy,Hungary,84,15,18
3,1950,Brazil,Uruguay,Brazil,88,13,22
4,1954,Switzerland,Germany FR,Hungary,140,16,26
